#                                       Predicting Telecom cutomer churn

## Churn is the largest problem faced by most of businesses. Customer churn means loss of clients or customers, this is also known as customer attrition. Cutomer churn analysis is a key business metric in telecom service providers, Internet service providers, Insurance firms etc. As the cost of retaining an existing customer is far less than acquiring a new customer( According to Harvard Business Review, it costs between 5 times and 25 times as much to find a new customer than to retain an existing one). 

## Preventing customer churn is an important function of business. By building predictive models using machine learning algorithms, we can predict whether a customer is willing to leave the carrier service. By identifying the customers who are in a risk of churning, network provider can provide better offers and services to retain their client.

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.sql import functions

#conf = SparkConf().setMaster("local").setAppName("SentenceEmotion")
#sc = SparkContext(conf = conf)

lines = sc.textFile("Telecomchurn.csv")

## Data

### We have used dataset from Kaggle. This data consists of various customers covering almost all the states in the united states. Each data entry involves the information about the customer which has features such as: 

### categorical variables 1) State, 2)International Plan 3) Voice mail plan 4) churn(Dependent variable)
### continuous variables 1) number vmail messages	2)total day minutes	3)total day calls	4)total day charge	5)total eve minutes	5)total eve calls	6)total eve charge	7)total night minutes	8)total night calls	9) total night charge	10)total intl minutes	11)total intl calls	12)total intl charge 13) customer service calls


### For the purpose of the project, the dependent variable is whether the customer will 'Churn' or not.

In [2]:
#checking top 5 lines
lines.take(5)

[u'state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn',
 u'KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10,3,2.7,1,FALSE',
 u'OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,FALSE',
 u'NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.3,162.6,104,7.32,12.2,5,3.29,0,FALSE',
 u'OH,84,408,375-9999,yes,no,0,299.4,71,50.9,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,FALSE']

In [5]:
header = lines.first()
sent = lines.filter(lambda x: x!=header)
sent.take(5)

Out[ 5 ]: 
['KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10,3,2.7,1,FALSE',
 'OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,FALSE',
 'NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.3,162.6,104,7.32,12.2,5,3.29,0,FALSE',
 'OH,84,408,375-9999,yes,no,0,299.4,71,50.9,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,FALSE',
 'OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,FALSE']

In [6]:
#creating dataframe
from numpy import array
cleanData =  sent.map(lambda line: [x for x in line.split(',')])
cleanData.take(5)

Out[ 12 ]: 
[['KS',
 '128',
 '415',
 '382-4657',
 'no',
 'yes',
 '25',
 '265.1',
 '110',
 '45.07',
 '197.4',
 '99',
 '16.78',
 '244.7',
 '91',
 '11.01',
 '10',
 '3',
 '2.7',
 '1',
 'FALSE'],
 ['OH',
 '107',
 '415',
 '371-7191',
 'no',
 'yes',
 '26',
 '161.6',
 '123',
 '27.47',
 '195.5',
 '103',
 '16.62',
 '254.4',
 '103',
 '11.45',
 '13.7',
 '3',
 '3.7',
 '1',
 'FALSE'],
 ['NJ',
 '137',
 '415',
 '358-1921',
 'no',
 'no',
 '0',
 '243.4',
 '114',
 '41.38',
 '121.2',
 '110',
 '10.3',
 '162.6',
 '104',
 '7.32',
 '12.2',
 '5',
 '3.29',
 '0',
 'FALSE'],
 ['OH',
 '84',
 '408',
 '375-9999',
 'yes',
 'no',
 '0',
 '299.4',
 '71',
 '50.9',
 '61.9',
 '88',
 '5.26',
 '196.9',
 '89',
 '8.86',
 '6.6',
 '7',
 '1.78',
 '2',
 'FALSE'],
 ['OK',
 '75',
 '415',
 '330-6626',
 'yes',
 'no',
 '0',
 '166.7',
 '113',
 '28.34',
 '148.3',
 '122',
 '12.61',
 '186.9',
 '121',
 '8.41',
 '10.1',
 '3',
 '2.73',
 '3',
 'FALSE']]

In [7]:
#splitting the file with ','
from numpy import array
data =  sent.map(lambda line: array([x for x in line.split(',')]))
data.take(5)

Out[ 13 ]: 
[array(['KS', '128', '415', '382-4657', 'no', 'yes', '25', '265.1', '110',
 '45.07', '197.4', '99', '16.78', '244.7', '91', '11.01', '10', '3',
 '2.7', '1', 'FALSE'], 
 dtype='<U8'),
 array(['OH', '107', '415', '371-7191', 'no', 'yes', '26', '161.6', '123',
 '27.47', '195.5', '103', '16.62', '254.4', '103', '11.45', '13.7',
 '3', '3.7', '1', 'FALSE'], 
 dtype='<U8'),
 array(['NJ', '137', '415', '358-1921', 'no', 'no', '0', '243.4', '114',
 '41.38', '121.2', '110', '10.3', '162.6', '104', '7.32', '12.2',
 '5', '3.29', '0', 'FALSE'], 
 dtype='<U8'),
 array(['OH', '84', '408', '375-9999', 'yes', 'no', '0', '299.4', '71',
 '50.9', '61.9', '88', '5.26', '196.9', '89', '8.86', '6.6', '7',
 '1.78', '2', 'FALSE'], 
 dtype='<U8'),
 array(['OK', '75', '415', '330-6626', 'yes', 'no', '0', '166.7', '113',
 '28.34', '148.3', '122', '12.61', '186.9', '121', '8.41', '10.1',
 '3', '2.73', '3', 'FALSE'], 
 dtype='<U8')]

In [8]:
#Creating features
X_features=data.map(lambda x: array([float(x[1]),float(x[2]),float(x[6]),float(x[7]),float(x[8]),float(x[9]),float(x[10]),float(x[11]),float(x[12]),float(x[13]),float(x[14]),float(x[15]),float(x[16]),float(x[17]),float(x[18]),float(x[19])]))
X_features.take(5)

Out[ 14 ]: 
[array([ 128. , 415. , 25. , 265.1 , 110. , 45.07, 197.4 ,
 99. , 16.78, 244.7 , 91. , 11.01, 10. , 3. ,
 2.7 , 1. ]),
 array([ 107. , 415. , 26. , 161.6 , 123. , 27.47, 195.5 ,
 103. , 16.62, 254.4 , 103. , 11.45, 13.7 , 3. ,
 3.7 , 1. ]),
 array([ 137. , 415. , 0. , 243.4 , 114. , 41.38, 121.2 ,
 110. , 10.3 , 162.6 , 104. , 7.32, 12.2 , 5. ,
 3.29, 0. ]),
 array([ 84. , 408. , 0. , 299.4 , 71. , 50.9 , 61.9 ,
 88. , 5.26, 196.9 , 89. , 8.86, 6.6 , 7. ,
 1.78, 2. ]),
 array([ 75. , 415. , 0. , 166.7 , 113. , 28.34, 148.3 ,
 122. , 12.61, 186.9 , 121. , 8.41, 10.1 , 3. ,
 2.73, 3. ])]

In [9]:
#importing Kmeans, KMeans modelling from MLlib and creating clusters.
from pyspark.mllib.clustering import KMeans, KMeansModel
clusters = KMeans.train(X_features, 2, maxIterations=150, runs=10,initializationMode="random",seed=42)

In [10]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [11]:
from math import sqrt
WSSSE = X_features.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 335816.45605647075

In [12]:
# Import the necessary modules 
from pyspark.sql import Row

# Map the RDD to a DF
df = cleanData.map(lambda x: Row(state=x[0], 
                              account_length=x[1], 
                              area_code=x[2],
                              phone_number=x[3],
                              international_plan=x[4],
                              voice_mail_plan=x[5], 
                              voice_mail_msgs=x[6],
                              day_mins=x[7],
                              day_calls=x[8],
                              day_charges=x[9],
                              eve_mins=x[10],
                              eve_calls=x[11],
                              eve_charges=x[12],
                              nit_mins=x[13],
                              nit_calls=x[14],
                              nit_charges=x[15],
                              intl_mins=x[16],
                              intl_calls=x[17],
                              intl_charges=x[18],
                              customer_service_calls=x[19],
                              churn=x[20])).toDF()

In [13]:
df.show()

+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+
account_length|area_code|churn|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|international_plan|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|phone_number|state|voice_mail_msgs|voice_mail_plan|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+
 128| 415|FALSE| 1| 110| 45.07| 265.1| 99| 16.78| 197.4| no| 3| 2.7| 10| 91| 11.01| 244.7| 382-4657| KS| 25| yes|
 107| 415|FALSE| 1| 123| 27.47| 161.6| 103| 16.62| 195.5| no| 3| 3.7| 13.7| 103| 11.45| 254.4| 371-7191| OH| 26| yes|
 137| 415|FALSE| 0| 114| 41.38| 243.4| 110| 10.3| 121.2| no| 5| 3.29| 12.2| 104| 7.32| 162.6| 358-1921| NJ| 0| no|
 84| 408|FALSE| 2| 71| 50.9| 299.4| 88| 5.26| 61.9| yes| 7| 1.78| 6.6| 89| 8.86| 196.9| 375-9999| OH| 0| no|
 75| 415|FALSE| 3| 113| 28.34| 166.7| 122| 12.61| 148.3| yes| 3| 2.73| 10.1| 121| 8.41| 186.9| 330-6626| OK| 0| no|
 118| 510|FALSE| 0| 98| 37.98| 223.4| 101| 18.75| 220.6| yes| 6| 1.7| 6.3| 118| 9.18| 203.9| 391-8027| AL| 0| no|
 121| 510|FALSE| 3| 88| 37.09| 218.2| 108| 29.62| 348.5| no| 7| 2.03| 7.5| 118| 9.57| 212.6| 355-9993| MA| 24| yes|
 147| 415|FALSE| 0| 79| 26.69| 157| 94| 8.76| 103.1| yes| 6| 1.92| 7.1| 96| 9.53| 211.8| 329-9001| MO| 0| no|
 117| 408|FALSE| 1| 97| 31.37| 184.5| 80| 29.89| 351.6| no| 4| 2.35| 8.7| 90| 9.71| 215.8| 335-4719| LA| 0| no|
 141| 415|FALSE| 0| 84| 43.96| 258.6| 111| 18.87| 222| yes| 5| 3.02| 11.2| 97| 14.69| 326.4| 330-8173| WV| 37| yes|
 65| 415| TRUE| 4| 137| 21.95| 129.1| 83| 19.42| 228.5| no| 6| 3.43| 12.7| 111| 9.4| 208.8| 329-6603| IN| 0| no|
 74| 415|FALSE| 0| 127| 31.91| 187.7| 148| 13.89| 163.4| no| 5| 2.46| 9.1| 94| 8.82| 196| 344-9403| RI| 0| no|
 168| 408|FALSE| 1| 96| 21.9| 128.8| 71| 8.92| 104.9| no| 2| 3.02| 11.2| 128| 6.35| 141.1| 363-1107| IA| 0| no|
 95| 510|FALSE| 3| 88| 26.62| 156.6| 75| 21.05| 247.6| no| 5| 3.32| 12.3| 115| 8.65| 192.3| 394-8006| MT| 0| no|
 62| 415|FALSE| 4| 70| 20.52| 120.7| 76| 26.11| 307.2| no| 6| 3.54| 13.1| 99| 9.14| 203| 366-9238| IA| 0| no|
 161| 415| TRUE| 4| 67| 56.59| 332.9| 97| 27.01| 317.8| no| 9| 1.46| 5.4| 128| 7.23| 160.6| 351-7269| NY| 0| no|
 85| 408|FALSE| 1| 139| 33.39| 196.4| 90| 23.88| 280.9| no| 4| 3.73| 13.8| 75| 4.02| 89.3| 350-8884| ID| 27| yes|
 93| 510|FALSE| 3| 114| 32.42| 190.7| 111| 18.55| 218.2| no| 3| 2.19| 8.1| 121| 5.83| 129.6| 386-2923| VT| 0| no|
 76| 510|FALSE| 1| 66| 32.25| 189.7| 65| 18.09| 212.8| no| 5| 2.7| 10| 108| 7.46| 165.7| 356-2992| VA| 33| yes|
 73| 415|FALSE| 1| 90| 38.15| 224.4| 88| 13.56| 159.5| no| 2| 3.51| 13| 74| 8.68| 192.8| 373-2782| TX| 0| no|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+
only showing top 20 rows

In [14]:
df.printSchema()

root
-- account_length: string (nullable = true)
-- area_code: string (nullable = true)
-- churn: string (nullable = true)
-- customer_service_calls: string (nullable = true)
-- day_calls: string (nullable = true)
-- day_charges: string (nullable = true)
-- day_mins: string (nullable = true)
-- eve_calls: string (nullable = true)
-- eve_charges: string (nullable = true)
-- eve_mins: string (nullable = true)
-- international_plan: string (nullable = true)
-- intl_calls: string (nullable = true)
-- intl_charges: string (nullable = true)
-- intl_mins: string (nullable = true)
-- nit_calls: string (nullable = true)
-- nit_charges: string (nullable = true)
-- nit_mins: string (nullable = true)
-- phone_number: string (nullable = true)
-- state: string (nullable = true)
-- voice_mail_msgs: string (nullable = true)
-- voice_mail_plan: string (nullable = true)

In [15]:
from pyspark.sql.types import *
df = df.withColumn("account_length", df["account_length"].cast(FloatType())) \
     .withColumn("voice_mail_msgs", df["voice_mail_msgs"].cast(FloatType())) .withColumn("day_mins", df["day_mins"].cast(FloatType())) .withColumn("day_calls", df["day_calls"].cast(FloatType())) .withColumn("day_charges", df["day_charges"].cast(FloatType())) .withColumn("eve_mins", df["eve_mins"].cast(FloatType())) .withColumn("eve_calls", df["eve_calls"].cast(FloatType())) .withColumn("eve_charges", df["eve_charges"].cast(FloatType())) .withColumn("nit_mins", df["nit_mins"].cast(FloatType())) .withColumn("nit_calls", df["nit_calls"].cast(FloatType())) .withColumn("nit_charges", df["nit_charges"].cast(FloatType())) .withColumn("intl_mins", df["intl_calls"].cast(FloatType())) .withColumn("intl_calls", df["intl_calls"].cast(FloatType())) .withColumn("intl_charges", df["intl_charges"].cast(FloatType())) .withColumn("customer_service_calls", df["customer_service_calls"].cast(FloatType()))
     #.withColumn("churn", df["churn"].cast(FloatType())) 

In [16]:
#Checking schema of variables
df.printSchema()

root
-- account_length: float (nullable = true)
-- area_code: string (nullable = true)
-- churn: string (nullable = true)
-- customer_service_calls: float (nullable = true)
-- day_calls: float (nullable = true)
-- day_charges: float (nullable = true)
-- day_mins: float (nullable = true)
-- eve_calls: float (nullable = true)
-- eve_charges: float (nullable = true)
-- eve_mins: float (nullable = true)
-- international_plan: string (nullable = true)
-- intl_calls: float (nullable = true)
-- intl_charges: float (nullable = true)
-- intl_mins: float (nullable = true)
-- nit_calls: float (nullable = true)
-- nit_charges: float (nullable = true)
-- nit_mins: float (nullable = true)
-- phone_number: string (nullable = true)
-- state: string (nullable = true)
-- voice_mail_msgs: float (nullable = true)
-- voice_mail_plan: string (nullable = true)

In [17]:
df.select('account_length','voice_mail_msgs').show(10)

+--------------+---------------+
account_length|voice_mail_msgs|
+--------------+---------------+
 128.0| 25.0|
 107.0| 26.0|
 137.0| 0.0|
 84.0| 0.0|
 75.0| 0.0|
 118.0| 0.0|
 121.0| 24.0|
 147.0| 0.0|
 117.0| 0.0|
 141.0| 37.0|
+--------------+---------------+
only showing top 10 rows

In [18]:
df.printSchema()

root
-- account_length: float (nullable = true)
-- area_code: string (nullable = true)
-- churn: string (nullable = true)
-- customer_service_calls: float (nullable = true)
-- day_calls: float (nullable = true)
-- day_charges: float (nullable = true)
-- day_mins: float (nullable = true)
-- eve_calls: float (nullable = true)
-- eve_charges: float (nullable = true)
-- eve_mins: float (nullable = true)
-- international_plan: string (nullable = true)
-- intl_calls: float (nullable = true)
-- intl_charges: float (nullable = true)
-- intl_mins: float (nullable = true)
-- nit_calls: float (nullable = true)
-- nit_charges: float (nullable = true)
-- nit_mins: float (nullable = true)
-- phone_number: string (nullable = true)
-- state: string (nullable = true)
-- voice_mail_msgs: float (nullable = true)
-- voice_mail_plan: string (nullable = true)

In [19]:
#Checking the count of churned and non-churned customers if they have international plan.
df.filter(df.international_plan == True).groupby(df.churn).count().show()

+-----+-----+
churn|count|
+-----+-----+
FALSE| 186|
 TRUE| 137|
+-----+-----+

### From the above results we could see that if the customer is having an international plan, number of customers who are willing to churn are more. 73% of the people with international plan are churned.

In [21]:
#Checking the same for the customers that churned that doesnot have international plan
df.filter(df.international_plan == False).groupby(df.churn).count().show()

+-----+-----+
churn|count|
+-----+-----+
FALSE| 2664|
 TRUE| 346|
+-----+-----+

### If the customer doesn't have an international plan, the percentage of customer's being churned are 12%. This can be interpretted in different ways. 

1) We could see that number of people with international plans are less compared to the number of people without international plan.
2) As we checked the percentage of churned to non-churned in both the cases. we can see that churn is more for international plan customers. We can assume that the international plans are not good or the service is not good that might caused the churn.

In [23]:
#Checkin the relation between customer service calls and churn
df.groupBy("churn").agg({'customer_service_calls': 'sum'}).show()

+-----+---------------------------+
churn|sum(customer_service_calls)|
+-----+---------------------------+
FALSE| 4132.0|
 TRUE| 1077.0|
+-----+---------------------------+

### We assume that the people that make more customer service calls would churn. As per the results we could see that the people with more customer service calls churned. As there might be issues with the service or the service calls couldn't help to resolve the issue.

In [25]:
# Exploring the customer churn with respect to area code.
df.filter(df.churn == True).groupby(df.area_code).count().orderBy('count', ascending=False).show()

+---------+-----+
area_code|count|
+---------+-----+
 415| 236|
 510| 125|
 408| 122|
+---------+-----+

### As per the above results we could see that the customers with area_code '415' churned the most with count of 256. followed by the area codes of '510' and '408'. Company could increase the service in that Area code or they might offer special packages for the customers with these area codes.

In [27]:
df.describe().show()

+-------+------------------+------------------+-----+----------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------+-----+------------------+---------------+
summary| account_length| area_code|churn|customer_service_calls| day_calls| day_charges| day_mins| eve_calls| eve_charges| eve_mins|international_plan| intl_calls| intl_charges| intl_mins| nit_calls| nit_charges| nit_mins|phone_number|state| voice_mail_msgs|voice_mail_plan|
+-------+------------------+------------------+-----+----------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------+-----+------------------+---------------+
 count| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333| 3333|
 mean|101.06480648064806|437.18241824182417| null| 1.5628562856285628|100.43564356435644|30.56230724989885|179.7750974000484|100.11431143114311|17.083540340032158|200.9803479203511| null|4.4794479447944795|2.7645814588593534|4.4794479447944795|100.10771077107711|9.039324928681985|200.87203718311895| null| null| 8.099009900990099| null|
 stddev| 39.82210592859559| 42.37129048560662| null| 1.3154910448664765|20.069084207300897| 9.25943456473607|54.46738896165808|19.922625293943113| 4.310667642830489|50.71384437339831| null| 2.461214270546093|0.7537726120916437| 2.461214270546093| 19.56860934605856|2.275872833666391| 50.5738471016932| null| null|13.688365372038598| null|
 min| 1.0| 408|FALSE| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| no| 0.0| 0.0| 0.0| 33.0| 1.04| 23.2| 327-1058| AK| 0.0| no|
 max| 243.0| 510| TRUE| 9.0| 165.0| 59.64| 350.8| 170.0| 30.91| 363.7| yes| 20.0| 5.4| 20.0| 175.0| 17.77| 395.0| 422-9964| WY| 51.0| yes|
+-------+------------------+------------------+-----+----------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------+-----+------------------+---------------+

In [28]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
indexer_state = StringIndexer(inputCol="state", outputCol="stateIndex")
indexed_state_df = indexer_state.fit(df).transform(df)
indexed_state_df.show()

+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+
account_length|area_code|churn|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|international_plan|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|phone_number|state|voice_mail_msgs|voice_mail_plan|stateIndex|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+
 128.0| 415|FALSE| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| no| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 382-4657| KS| 25.0| yes| 17.0|
 107.0| 415|FALSE| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| no| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 371-7191| OH| 26.0| yes| 5.0|
 137.0| 415|FALSE| 0.0| 114.0| 41.38| 243.4| 110.0| 10.3| 121.2| no| 5.0| 3.29| 5.0| 104.0| 7.32| 162.6| 358-1921| NJ| 0.0| no| 18.0|
 84.0| 408|FALSE| 2.0| 71.0| 50.9| 299.4| 88.0| 5.26| 61.9| yes| 7.0| 1.78| 7.0| 89.0| 8.86| 196.9| 375-9999| OH| 0.0| no| 5.0|
 75.0| 415|FALSE| 3.0| 113.0| 28.34| 166.7| 122.0| 12.61| 148.3| yes| 3.0| 2.73| 3.0| 121.0| 8.41| 186.9| 330-6626| OK| 0.0| no| 34.0|
 118.0| 510|FALSE| 0.0| 98.0| 37.98| 223.4| 101.0| 18.75| 220.6| yes| 6.0| 1.7| 6.0| 118.0| 9.18| 203.9| 391-8027| AL| 0.0| no| 3.0|
 121.0| 510|FALSE| 3.0| 88.0| 37.09| 218.2| 108.0| 29.62| 348.5| no| 7.0| 2.03| 7.0| 118.0| 9.57| 212.6| 355-9993| MA| 24.0| yes| 24.0|
 147.0| 415|FALSE| 0.0| 79.0| 26.69| 157.0| 94.0| 8.76| 103.1| yes| 6.0| 1.92| 6.0| 96.0| 9.53| 211.8| 329-9001| MO| 0.0| no| 28.0|
 117.0| 408|FALSE| 1.0| 97.0| 31.37| 184.5| 80.0| 29.89| 351.6| no| 4.0| 2.35| 4.0| 90.0| 9.71| 215.8| 335-4719| LA| 0.0| no| 47.0|
 141.0| 415|FALSE| 0.0| 84.0| 43.96| 258.6| 111.0| 18.87| 222.0| yes| 5.0| 3.02| 5.0| 97.0| 14.69| 326.4| 330-8173| WV| 37.0| yes| 0.0|
 65.0| 415| TRUE| 4.0| 137.0| 21.95| 129.1| 83.0| 19.42| 228.5| no| 6.0| 3.43| 6.0| 111.0| 9.4| 208.8| 329-6603| IN| 0.0| no| 15.0|
 74.0| 415|FALSE| 0.0| 127.0| 31.91| 187.7| 148.0| 13.89| 163.4| no| 5.0| 2.46| 5.0| 94.0| 8.82| 196.0| 344-9403| RI| 0.0| no| 25.0|
 168.0| 408|FALSE| 1.0| 96.0| 21.9| 128.8| 71.0| 8.92| 104.9| no| 2.0| 3.02| 2.0| 128.0| 6.35| 141.1| 363-1107| IA| 0.0| no| 49.0|
 95.0| 510|FALSE| 3.0| 88.0| 26.62| 156.6| 75.0| 21.05| 247.6| no| 5.0| 3.32| 5.0| 115.0| 8.65| 192.3| 394-8006| MT| 0.0| no| 19.0|
 62.0| 415|FALSE| 4.0| 70.0| 20.52| 120.7| 76.0| 26.11| 307.2| no| 6.0| 3.54| 6.0| 99.0| 9.14| 203.0| 366-9238| IA| 0.0| no| 49.0|
 161.0| 415| TRUE| 4.0| 67.0| 56.59| 332.9| 97.0| 27.01| 317.8| no| 9.0| 1.46| 9.0| 128.0| 7.23| 160.6| 351-7269| NY| 0.0| no| 2.0|
 85.0| 408|FALSE| 1.0| 139.0| 33.39| 196.4| 90.0| 23.88| 280.9| no| 4.0| 3.73| 4.0| 75.0| 4.02| 89.3| 350-8884| ID| 27.0| yes| 10.0|
 93.0| 510|FALSE| 3.0| 114.0| 32.42| 190.7| 111.0| 18.55| 218.2| no| 3.0| 2.19| 3.0| 121.0| 5.83| 129.6| 386-2923| VT| 0.0| no| 11.0|
 76.0| 510|FALSE| 1.0| 66.0| 32.25| 189.7| 65.0| 18.09| 212.8| no| 5.0| 2.7| 5.0| 108.0| 7.46| 165.7| 356-2992| VA| 33.0| yes| 8.0|
 73.0| 415|FALSE| 1.0| 90.0| 38.15| 224.4| 88.0| 13.56| 159.5| no| 2.0| 3.51| 2.0| 74.0| 8.68| 192.8| 373-2782| TX| 0.0| no| 13.0|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+
only showing top 20 rows

In [29]:
#Further expanding the same logic to other categorical columns.
from pyspark.ml.feature import StringIndexer
indexer_intplan = StringIndexer(inputCol="international_plan", outputCol="intlplanIndex")
indexed_state_intplan_df = indexer_intplan.fit(indexed_state_df).transform(indexed_state_df)
indexed_state_intplan_df.show()

+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+
account_length|area_code|churn|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|international_plan|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|phone_number|state|voice_mail_msgs|voice_mail_plan|stateIndex|intlplanIndex|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+
 128.0| 415|FALSE| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| no| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 382-4657| KS| 25.0| yes| 17.0| 0.0|
 107.0| 415|FALSE| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| no| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 371-7191| OH| 26.0| yes| 5.0| 0.0|
 137.0| 415|FALSE| 0.0| 114.0| 41.38| 243.4| 110.0| 10.3| 121.2| no| 5.0| 3.29| 5.0| 104.0| 7.32| 162.6| 358-1921| NJ| 0.0| no| 18.0| 0.0|
 84.0| 408|FALSE| 2.0| 71.0| 50.9| 299.4| 88.0| 5.26| 61.9| yes| 7.0| 1.78| 7.0| 89.0| 8.86| 196.9| 375-9999| OH| 0.0| no| 5.0| 1.0|
 75.0| 415|FALSE| 3.0| 113.0| 28.34| 166.7| 122.0| 12.61| 148.3| yes| 3.0| 2.73| 3.0| 121.0| 8.41| 186.9| 330-6626| OK| 0.0| no| 34.0| 1.0|
 118.0| 510|FALSE| 0.0| 98.0| 37.98| 223.4| 101.0| 18.75| 220.6| yes| 6.0| 1.7| 6.0| 118.0| 9.18| 203.9| 391-8027| AL| 0.0| no| 3.0| 1.0|
 121.0| 510|FALSE| 3.0| 88.0| 37.09| 218.2| 108.0| 29.62| 348.5| no| 7.0| 2.03| 7.0| 118.0| 9.57| 212.6| 355-9993| MA| 24.0| yes| 24.0| 0.0|
 147.0| 415|FALSE| 0.0| 79.0| 26.69| 157.0| 94.0| 8.76| 103.1| yes| 6.0| 1.92| 6.0| 96.0| 9.53| 211.8| 329-9001| MO| 0.0| no| 28.0| 1.0|
 117.0| 408|FALSE| 1.0| 97.0| 31.37| 184.5| 80.0| 29.89| 351.6| no| 4.0| 2.35| 4.0| 90.0| 9.71| 215.8| 335-4719| LA| 0.0| no| 47.0| 0.0|
 141.0| 415|FALSE| 0.0| 84.0| 43.96| 258.6| 111.0| 18.87| 222.0| yes| 5.0| 3.02| 5.0| 97.0| 14.69| 326.4| 330-8173| WV| 37.0| yes| 0.0| 1.0|
 65.0| 415| TRUE| 4.0| 137.0| 21.95| 129.1| 83.0| 19.42| 228.5| no| 6.0| 3.43| 6.0| 111.0| 9.4| 208.8| 329-6603| IN| 0.0| no| 15.0| 0.0|
 74.0| 415|FALSE| 0.0| 127.0| 31.91| 187.7| 148.0| 13.89| 163.4| no| 5.0| 2.46| 5.0| 94.0| 8.82| 196.0| 344-9403| RI| 0.0| no| 25.0| 0.0|
 168.0| 408|FALSE| 1.0| 96.0| 21.9| 128.8| 71.0| 8.92| 104.9| no| 2.0| 3.02| 2.0| 128.0| 6.35| 141.1| 363-1107| IA| 0.0| no| 49.0| 0.0|
 95.0| 510|FALSE| 3.0| 88.0| 26.62| 156.6| 75.0| 21.05| 247.6| no| 5.0| 3.32| 5.0| 115.0| 8.65| 192.3| 394-8006| MT| 0.0| no| 19.0| 0.0|
 62.0| 415|FALSE| 4.0| 70.0| 20.52| 120.7| 76.0| 26.11| 307.2| no| 6.0| 3.54| 6.0| 99.0| 9.14| 203.0| 366-9238| IA| 0.0| no| 49.0| 0.0|
 161.0| 415| TRUE| 4.0| 67.0| 56.59| 332.9| 97.0| 27.01| 317.8| no| 9.0| 1.46| 9.0| 128.0| 7.23| 160.6| 351-7269| NY| 0.0| no| 2.0| 0.0|
 85.0| 408|FALSE| 1.0| 139.0| 33.39| 196.4| 90.0| 23.88| 280.9| no| 4.0| 3.73| 4.0| 75.0| 4.02| 89.3| 350-8884| ID| 27.0| yes| 10.0| 0.0|
 93.0| 510|FALSE| 3.0| 114.0| 32.42| 190.7| 111.0| 18.55| 218.2| no| 3.0| 2.19| 3.0| 121.0| 5.83| 129.6| 386-2923| VT| 0.0| no| 11.0| 0.0|
 76.0| 510|FALSE| 1.0| 66.0| 32.25| 189.7| 65.0| 18.09| 212.8| no| 5.0| 2.7| 5.0| 108.0| 7.46| 165.7| 356-2992| VA| 33.0| yes| 8.0| 0.0|
 73.0| 415|FALSE| 1.0| 90.0| 38.15| 224.4| 88.0| 13.56| 159.5| no| 2.0| 3.51| 2.0| 74.0| 8.68| 192.8| 373-2782| TX| 0.0| no| 13.0| 0.0|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+
only showing top 20 rows

In [30]:
#Expanding indexes to further to dependent variables.
from pyspark.ml.feature import StringIndexer
indexer_churn = StringIndexer(inputCol="churn", outputCol="churnIndex")
indexed_c_s_ip_df = indexer_churn.fit(indexed_state_intplan_df).transform(indexed_state_intplan_df)
indexed_c_s_ip_df.show()

+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+----------+
account_length|area_code|churn|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|international_plan|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|phone_number|state|voice_mail_msgs|voice_mail_plan|stateIndex|intlplanIndex|churnIndex|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+----------+
 128.0| 415|FALSE| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| no| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 382-4657| KS| 25.0| yes| 17.0| 0.0| 0.0|
 107.0| 415|FALSE| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| no| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 371-7191| OH| 26.0| yes| 5.0| 0.0| 0.0|
 137.0| 415|FALSE| 0.0| 114.0| 41.38| 243.4| 110.0| 10.3| 121.2| no| 5.0| 3.29| 5.0| 104.0| 7.32| 162.6| 358-1921| NJ| 0.0| no| 18.0| 0.0| 0.0|
 84.0| 408|FALSE| 2.0| 71.0| 50.9| 299.4| 88.0| 5.26| 61.9| yes| 7.0| 1.78| 7.0| 89.0| 8.86| 196.9| 375-9999| OH| 0.0| no| 5.0| 1.0| 0.0|
 75.0| 415|FALSE| 3.0| 113.0| 28.34| 166.7| 122.0| 12.61| 148.3| yes| 3.0| 2.73| 3.0| 121.0| 8.41| 186.9| 330-6626| OK| 0.0| no| 34.0| 1.0| 0.0|
 118.0| 510|FALSE| 0.0| 98.0| 37.98| 223.4| 101.0| 18.75| 220.6| yes| 6.0| 1.7| 6.0| 118.0| 9.18| 203.9| 391-8027| AL| 0.0| no| 3.0| 1.0| 0.0|
 121.0| 510|FALSE| 3.0| 88.0| 37.09| 218.2| 108.0| 29.62| 348.5| no| 7.0| 2.03| 7.0| 118.0| 9.57| 212.6| 355-9993| MA| 24.0| yes| 24.0| 0.0| 0.0|
 147.0| 415|FALSE| 0.0| 79.0| 26.69| 157.0| 94.0| 8.76| 103.1| yes| 6.0| 1.92| 6.0| 96.0| 9.53| 211.8| 329-9001| MO| 0.0| no| 28.0| 1.0| 0.0|
 117.0| 408|FALSE| 1.0| 97.0| 31.37| 184.5| 80.0| 29.89| 351.6| no| 4.0| 2.35| 4.0| 90.0| 9.71| 215.8| 335-4719| LA| 0.0| no| 47.0| 0.0| 0.0|
 141.0| 415|FALSE| 0.0| 84.0| 43.96| 258.6| 111.0| 18.87| 222.0| yes| 5.0| 3.02| 5.0| 97.0| 14.69| 326.4| 330-8173| WV| 37.0| yes| 0.0| 1.0| 0.0|
 65.0| 415| TRUE| 4.0| 137.0| 21.95| 129.1| 83.0| 19.42| 228.5| no| 6.0| 3.43| 6.0| 111.0| 9.4| 208.8| 329-6603| IN| 0.0| no| 15.0| 0.0| 1.0|
 74.0| 415|FALSE| 0.0| 127.0| 31.91| 187.7| 148.0| 13.89| 163.4| no| 5.0| 2.46| 5.0| 94.0| 8.82| 196.0| 344-9403| RI| 0.0| no| 25.0| 0.0| 0.0|
 168.0| 408|FALSE| 1.0| 96.0| 21.9| 128.8| 71.0| 8.92| 104.9| no| 2.0| 3.02| 2.0| 128.0| 6.35| 141.1| 363-1107| IA| 0.0| no| 49.0| 0.0| 0.0|
 95.0| 510|FALSE| 3.0| 88.0| 26.62| 156.6| 75.0| 21.05| 247.6| no| 5.0| 3.32| 5.0| 115.0| 8.65| 192.3| 394-8006| MT| 0.0| no| 19.0| 0.0| 0.0|
 62.0| 415|FALSE| 4.0| 70.0| 20.52| 120.7| 76.0| 26.11| 307.2| no| 6.0| 3.54| 6.0| 99.0| 9.14| 203.0| 366-9238| IA| 0.0| no| 49.0| 0.0| 0.0|
 161.0| 415| TRUE| 4.0| 67.0| 56.59| 332.9| 97.0| 27.01| 317.8| no| 9.0| 1.46| 9.0| 128.0| 7.23| 160.6| 351-7269| NY| 0.0| no| 2.0| 0.0| 1.0|
 85.0| 408|FALSE| 1.0| 139.0| 33.39| 196.4| 90.0| 23.88| 280.9| no| 4.0| 3.73| 4.0| 75.0| 4.02| 89.3| 350-8884| ID| 27.0| yes| 10.0| 0.0| 0.0|
 93.0| 510|FALSE| 3.0| 114.0| 32.42| 190.7| 111.0| 18.55| 218.2| no| 3.0| 2.19| 3.0| 121.0| 5.83| 129.6| 386-2923| VT| 0.0| no| 11.0| 0.0| 0.0|
 76.0| 510|FALSE| 1.0| 66.0| 32.25| 189.7| 65.0| 18.09| 212.8| no| 5.0| 2.7| 5.0| 108.0| 7.46| 165.7| 356-2992| VA| 33.0| yes| 8.0| 0.0| 0.0|
 73.0| 415|FALSE| 1.0| 90.0| 38.15| 224.4| 88.0| 13.56| 159.5| no| 2.0| 3.51| 2.0| 74.0| 8.68| 192.8| 373-2782| TX| 0.0| no| 13.0| 0.0| 0.0|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------

In [31]:
from pyspark.ml.feature import StringIndexer
indexer_vp = StringIndexer(inputCol="voice_mail_plan", outputCol="voiceplanIndex")
indexed_c_s_ip_vp_df = indexer_vp.fit(indexed_c_s_ip_df).transform(indexed_c_s_ip_df)
indexed_c_s_ip_vp_df.show()

+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+----------+--------------+
account_length|area_code|churn|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|international_plan|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|phone_number|state|voice_mail_msgs|voice_mail_plan|stateIndex|intlplanIndex|churnIndex|voiceplanIndex|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+----------+--------------+
 128.0| 415|FALSE| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| no| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 382-4657| KS| 25.0| yes| 17.0| 0.0| 0.0| 1.0|
 107.0| 415|FALSE| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| no| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 371-7191| OH| 26.0| yes| 5.0| 0.0| 0.0| 1.0|
 137.0| 415|FALSE| 0.0| 114.0| 41.38| 243.4| 110.0| 10.3| 121.2| no| 5.0| 3.29| 5.0| 104.0| 7.32| 162.6| 358-1921| NJ| 0.0| no| 18.0| 0.0| 0.0| 0.0|
 84.0| 408|FALSE| 2.0| 71.0| 50.9| 299.4| 88.0| 5.26| 61.9| yes| 7.0| 1.78| 7.0| 89.0| 8.86| 196.9| 375-9999| OH| 0.0| no| 5.0| 1.0| 0.0| 0.0|
 75.0| 415|FALSE| 3.0| 113.0| 28.34| 166.7| 122.0| 12.61| 148.3| yes| 3.0| 2.73| 3.0| 121.0| 8.41| 186.9| 330-6626| OK| 0.0| no| 34.0| 1.0| 0.0| 0.0|
 118.0| 510|FALSE| 0.0| 98.0| 37.98| 223.4| 101.0| 18.75| 220.6| yes| 6.0| 1.7| 6.0| 118.0| 9.18| 203.9| 391-8027| AL| 0.0| no| 3.0| 1.0| 0.0| 0.0|
 121.0| 510|FALSE| 3.0| 88.0| 37.09| 218.2| 108.0| 29.62| 348.5| no| 7.0| 2.03| 7.0| 118.0| 9.57| 212.6| 355-9993| MA| 24.0| yes| 24.0| 0.0| 0.0| 1.0|
 147.0| 415|FALSE| 0.0| 79.0| 26.69| 157.0| 94.0| 8.76| 103.1| yes| 6.0| 1.92| 6.0| 96.0| 9.53| 211.8| 329-9001| MO| 0.0| no| 28.0| 1.0| 0.0| 0.0|
 117.0| 408|FALSE| 1.0| 97.0| 31.37| 184.5| 80.0| 29.89| 351.6| no| 4.0| 2.35| 4.0| 90.0| 9.71| 215.8| 335-4719| LA| 0.0| no| 47.0| 0.0| 0.0| 0.0|
 141.0| 415|FALSE| 0.0| 84.0| 43.96| 258.6| 111.0| 18.87| 222.0| yes| 5.0| 3.02| 5.0| 97.0| 14.69| 326.4| 330-8173| WV| 37.0| yes| 0.0| 1.0| 0.0| 1.0|
 65.0| 415| TRUE| 4.0| 137.0| 21.95| 129.1| 83.0| 19.42| 228.5| no| 6.0| 3.43| 6.0| 111.0| 9.4| 208.8| 329-6603| IN| 0.0| no| 15.0| 0.0| 1.0| 0.0|
 74.0| 415|FALSE| 0.0| 127.0| 31.91| 187.7| 148.0| 13.89| 163.4| no| 5.0| 2.46| 5.0| 94.0| 8.82| 196.0| 344-9403| RI| 0.0| no| 25.0| 0.0| 0.0| 0.0|
 168.0| 408|FALSE| 1.0| 96.0| 21.9| 128.8| 71.0| 8.92| 104.9| no| 2.0| 3.02| 2.0| 128.0| 6.35| 141.1| 363-1107| IA| 0.0| no| 49.0| 0.0| 0.0| 0.0|
 95.0| 510|FALSE| 3.0| 88.0| 26.62| 156.6| 75.0| 21.05| 247.6| no| 5.0| 3.32| 5.0| 115.0| 8.65| 192.3| 394-8006| MT| 0.0| no| 19.0| 0.0| 0.0| 0.0|
 62.0| 415|FALSE| 4.0| 70.0| 20.52| 120.7| 76.0| 26.11| 307.2| no| 6.0| 3.54| 6.0| 99.0| 9.14| 203.0| 366-9238| IA| 0.0| no| 49.0| 0.0| 0.0| 0.0|
 161.0| 415| TRUE| 4.0| 67.0| 56.59| 332.9| 97.0| 27.01| 317.8| no| 9.0| 1.46| 9.0| 128.0| 7.23| 160.6| 351-7269| NY| 0.0| no| 2.0| 0.0| 1.0| 0.0|
 85.0| 408|FALSE| 1.0| 139.0| 33.39| 196.4| 90.0| 23.88| 280.9| no| 4.0| 3.73| 4.0| 75.0| 4.02| 89.3| 350-8884| ID| 27.0| yes| 10.0| 0.0| 0.0| 1.0|
 93.0| 510|FALSE| 3.0| 114.0| 32.42| 190.7| 111.0| 18.55| 218.2| no| 3.0| 2.19| 3.0| 121.0| 5.83| 129.6| 386-2923| VT| 0.0| no| 11.0| 0.0| 0.0| 0.0|
 76.0| 510|FALSE| 1.0| 66.0| 32.25| 189.7| 65.0| 18.09| 212.8| no| 5.0| 2.7| 5.0| 108.0| 7.46| 165.7| 356-2992| VA| 33.0| yes| 8.0| 0.0| 0.0| 1.0|
 73.0| 415|FALSE| 1.0| 90.0| 38.15| 224.4| 88.0| 13.56| 159.5| no| 2.0| 3.51| 2.0| 74.0| 8.68| 192.8| 373-2782| TX| 0.0| no| 13.0| 0.0| 0.0| 0.0|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+--------------

In [32]:
#Applying onehot encoder to convert all the categorical variables into sparse matrices.
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["stateIndex","intlplanIndex","voiceplanIndex","churnIndex"],
                                 outputCols=["stateVec1","intlplanvec1","voiceplanvec1","churnvec1"])
model = encoder.fit(indexed_c_s_ip_vp_df)
encoded = model.transform(indexed_c_s_ip_vp_df)
encoded.show()

+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+----------+--------------+---------------+-------------+-------------+-------------+
account_length|area_code|churn|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|international_plan|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|phone_number|state|voice_mail_msgs|voice_mail_plan|stateIndex|intlplanIndex|churnIndex|voiceplanIndex| stateVec1| intlplanvec1|voiceplanvec1| churnvec1|
+--------------+---------+-----+----------------------+---------+-----------+--------+---------+-----------+--------+------------------+----------+------------+---------+---------+-----------+--------+------------+-----+---------------+---------------+----------+-------------+----------+--------------+---------------+-------------+-------------+-------------+
 128.0| 415|FALSE| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| no| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 382-4657| KS| 25.0| yes| 17.0| 0.0| 0.0| 1.0|(50,[17],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
 107.0| 415|FALSE| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| no| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 371-7191| OH| 26.0| yes| 5.0| 0.0| 0.0| 1.0| (50,[5],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
 137.0| 415|FALSE| 0.0| 114.0| 41.38| 243.4| 110.0| 10.3| 121.2| no| 5.0| 3.29| 5.0| 104.0| 7.32| 162.6| 358-1921| NJ| 0.0| no| 18.0| 0.0| 0.0| 0.0|(50,[18],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
 84.0| 408|FALSE| 2.0| 71.0| 50.9| 299.4| 88.0| 5.26| 61.9| yes| 7.0| 1.78| 7.0| 89.0| 8.86| 196.9| 375-9999| OH| 0.0| no| 5.0| 1.0| 0.0| 0.0| (50,[5],[1.0])| (1,[],[])|(1,[0],[1.0])|(1,[0],[1.0])|
 75.0| 415|FALSE| 3.0| 113.0| 28.34| 166.7| 122.0| 12.61| 148.3| yes| 3.0| 2.73| 3.0| 121.0| 8.41| 186.9| 330-6626| OK| 0.0| no| 34.0| 1.0| 0.0| 0.0|(50,[34],[1.0])| (1,[],[])|(1,[0],[1.0])|(1,[0],[1.0])|
 118.0| 510|FALSE| 0.0| 98.0| 37.98| 223.4| 101.0| 18.75| 220.6| yes| 6.0| 1.7| 6.0| 118.0| 9.18| 203.9| 391-8027| AL| 0.0| no| 3.0| 1.0| 0.0| 0.0| (50,[3],[1.0])| (1,[],[])|(1,[0],[1.0])|(1,[0],[1.0])|
 121.0| 510|FALSE| 3.0| 88.0| 37.09| 218.2| 108.0| 29.62| 348.5| no| 7.0| 2.03| 7.0| 118.0| 9.57| 212.6| 355-9993| MA| 24.0| yes| 24.0| 0.0| 0.0| 1.0|(50,[24],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
 147.0| 415|FALSE| 0.0| 79.0| 26.69| 157.0| 94.0| 8.76| 103.1| yes| 6.0| 1.92| 6.0| 96.0| 9.53| 211.8| 329-9001| MO| 0.0| no| 28.0| 1.0| 0.0| 0.0|(50,[28],[1.0])| (1,[],[])|(1,[0],[1.0])|(1,[0],[1.0])|
 117.0| 408|FALSE| 1.0| 97.0| 31.37| 184.5| 80.0| 29.89| 351.6| no| 4.0| 2.35| 4.0| 90.0| 9.71| 215.8| 335-4719| LA| 0.0| no| 47.0| 0.0| 0.0| 0.0|(50,[47],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
 141.0| 415|FALSE| 0.0| 84.0| 43.96| 258.6| 111.0| 18.87| 222.0| yes| 5.0| 3.02| 5.0| 97.0| 14.69| 326.4| 330-8173| WV| 37.0| yes| 0.0| 1.0| 0.0| 1.0| (50,[0],[1.0])| (1,[],[])| (1,[],[])|(1,[0],[1.0])|
 65.0| 415| TRUE| 4.0| 137.0| 21.95| 129.1| 83.0| 19.42| 228.5| no| 6.0| 3.43| 6.0| 111.0| 9.4| 208.8| 329-6603| IN| 0.0| no| 15.0| 0.0| 1.0| 0.0|(50,[15],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])| (1,[],[])|
 74.0| 415|FALSE| 0.0| 127.0| 31.91| 187.7| 148.0| 13.89| 163.4| no| 5.0| 2.46| 5.0| 94.0| 8.82| 196.0| 344-9403| RI| 0.0| no| 25.0| 0.0| 0.0| 0.0|(50,[25],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
 168.0| 408|FALSE| 1.0| 96.0| 21.9| 128.8| 71.0| 8.92| 104.9| no| 2.0| 3.02| 2.0| 128.0| 6.35| 141.1| 363-1107| IA| 0.0| no| 49.0| 0.0| 0.0| 0.0|(50,[49],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
 95.0| 510|FALSE| 3.0| 88.0| 26.62| 156.6| 75.0| 21.05| 247.6| no| 5.0| 3.32| 5.0| 115.0| 8.65| 192.3| 394-8006| MT| 0.0| no| 19.0| 0.0| 0.0| 0.0|(50,[19],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
 62.0| 415|FALSE| 4.0| 70.0| 20.52| 120.7| 76.0| 26.11| 307.2| no|

In [33]:
# Assigning the variables to apply encoder.
df1=encoded.select([c for c in encoded.columns if c not in {'state','area_code','phone_number','churn','stateIndex','intlplanIndex','voiceplanIndex','international_plan','voice_mail_plan'}])

In [34]:
df1.show(2)

+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+
account_length|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|voice_mail_msgs|churnIndex| stateVec1| intlplanvec1|voiceplanvec1| churnvec1|
+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+
 128.0| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 25.0| 0.0|(50,[17],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
 107.0| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 26.0| 0.0| (50,[5],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+
only showing top 2 rows

In [35]:
#Selecting the required columns for features.
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["account_length","customer_service_calls","day_calls","day_charges","day_mins","eve_calls","eve_charges","eve_mins","intl_calls","intl_charges","intl_mins","nit_calls","nit_charges","nit_mins","voice_mail_msgs","stateVec1","intlplanvec1","voiceplanvec1"],
    outputCol="features")
output = assembler.transform(df1)
output.show(2)
#output.select("features").show(truncate=False)

+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+--------------------+
account_length|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|voice_mail_msgs|churnIndex| stateVec1| intlplanvec1|voiceplanvec1| churnvec1| features|
+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+--------------------+
 128.0| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 25.0| 0.0|(50,[17],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|(67,[0,1,2,3,4,5,...|
 107.0| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 26.0| 0.0| (50,[5],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|(67,[0,1,2,3,4,5,...|
+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+--------------------+
only showing top 2 rows

In [36]:
df1.show(2)

+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+
account_length|customer_service_calls|day_calls|day_charges|day_mins|eve_calls|eve_charges|eve_mins|intl_calls|intl_charges|intl_mins|nit_calls|nit_charges|nit_mins|voice_mail_msgs|churnIndex| stateVec1| intlplanvec1|voiceplanvec1| churnvec1|
+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+
 128.0| 1.0| 110.0| 45.07| 265.1| 99.0| 16.78| 197.4| 3.0| 2.7| 3.0| 91.0| 11.01| 244.7| 25.0| 0.0|(50,[17],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
 107.0| 1.0| 123.0| 27.47| 161.6| 103.0| 16.62| 195.5| 3.0| 3.7| 3.0| 103.0| 11.45| 254.4| 26.0| 0.0| (50,[5],[1.0])|(1,[0],[1.0])| (1,[],[])|(1,[0],[1.0])|
+--------------+----------------------+---------+-----------+--------+---------+-----------+--------+----------+------------+---------+---------+-----------+--------+---------------+----------+---------------+-------------+-------------+-------------+
only showing top 2 rows

In [37]:
#Applying features and lables to provide the same to model.
df_classifier = output.selectExpr("features as features","churnIndex as label")
df_classifier.show(2)

+--------------------+-----+
 features|label|
+--------------------+-----+
(67,[0,1,2,3,4,5,...| 0.0|
(67,[0,1,2,3,4,5,...| 0.0|
+--------------------+-----+
only showing top 2 rows

In [38]:
df_log_reg = output.selectExpr("features as features","churnvec1 as label")
df_log_reg.show(2)

+--------------------+-------------+
 features| label|
+--------------------+-------------+
(67,[0,1,2,3,4,5,...|(1,[0],[1.0])|
(67,[0,1,2,3,4,5,...|(1,[0],[1.0])|
+--------------------+-------------+
only showing top 2 rows

In [39]:
df_log_reg.select("features").show(1)

+--------------------+
 features|
+--------------------+
(67,[0,1,2,3,4,5,...|
+--------------------+
only showing top 1 row

In [40]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.5)
# Fit the model
lrModel = lr.fit(df_classifier)
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(df_classifier)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

Coefficients: (67,[],[])
Intercept: -1.7750576196101235
Multinomial coefficients: 2 X 67 CSRMatrix

Multinomial intercepts: [0.887528802698,-0.887528802698]

In [41]:
#Evaluation metrices
trainingSummary = lrModel.summary
# for multiclass, we can inspect metrics on a per-label basis

print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s\nareaUnderROC: %s "
      % (trainingSummary.accuracy, trainingSummary.weightedFalsePositiveRate, trainingSummary.weightedTruePositiveRate, trainingSummary.weightedFMeasure(), trainingSummary.weightedPrecision, trainingSummary.weightedRecall,trainingSummary.areaUnderROC ))

Accuracy: 0.8550855085508551
FPR: 0.8550855085508551
TPR: 0.8550855085508551
F-measure: 0.7882884358304827
Precision: 0.7311712269336745
Recall: 0.8550855085508551
areaUnderROC: 0.5

In [42]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_classifier.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

# Chain indexers and tree in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = dt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
#predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g\naccuracy = %g " % ((1.0 - accuracy) ,accuracy))

# summary only
print(model)

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [2, 4, 6])
             .addGrid(dt.maxBins, [20, 60])
             .build())
crossval = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

cvModel = crossval.fit(trainingData)
prediction = cvModel.transform(testData)
evaluator.evaluate(predictions)

Test Error = 0.0607287
accuracy = 0.939271 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_dd370542e896) of depth 5 with 53 nodes
Test Area Under ROC: 0.45086643792495595
 Out[ 59 ]: 0.45086643792495595

In [43]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
model = rf.fit(trainingData)
# Make predictions.
predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g\naccuracy = %g " % ((1.0 - accuracy) ,accuracy))
treeModel = model
# summary only
print(treeModel)
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Error = 0.124494
accuracy = 0.875506 
RandomForestClassificationModel (uid=RandomForestClassifier_2ba0160ed4eb) with 10 trees
Test Area Under ROC: 0.9074894547972792

In [44]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=50)

# Chain indexers and GBT in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = gbt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g\naccuracy = %g " % ((1.0 - accuracy) ,accuracy))

print(model)  # summary only

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Error = 0.0465587
accuracy = 0.953441 
GBTClassificationModel (uid=GBTClassifier_1dc6a78a94e2) with 50 trees
Test Area Under ROC: 0.9386528897003251

In [45]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)
predictions = cvModel.transform(testData)
evaluator.evaluate(predictions)

Out[ 62 ]: 0.9325264417023421

In [46]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa
# Load training data
#data = spark.read.format("libsvm")\.load("data/mllib/sample_multiclass_classification_data.txt")

# Split the data into train and test
splits = df_classifier.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = [4, 5, 4, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(trainingData)

# compute accuracy on the test set
result = model.transform(testData)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g\naccuracy = %g " % ((1.0 - accuracy) ,accuracy))
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Error = 0.0475709
accuracy = 0.952429 
Test Area Under ROC: 0.9325264417023418

In [47]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
scaler_model = scaler.fit(df_classifier)
scaled_data = scaler_model.transform(df_classifier)
type(scaled_data)

Out[ 64 ]: pyspark.sql.dataframe.DataFrame

In [48]:
scaled_data.show()

+--------------------+-----+--------------------+
 features|label| scaledFeatures|
+--------------------+-----+--------------------+
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,2,3,4,5,6,...| 0.0|(67,[0,2,3,4,5,6,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,2,3,4,5,6,...| 0.0|(67,[0,2,3,4,5,6,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,2,3,4,5,6,...| 0.0|(67,[0,2,3,4,5,6,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,2,3,4,5,6,...| 0.0|(67,[0,2,3,4,5,6,...|
(67,[0,1,2,3,4,5,...| 1.0|(67,[0,1,2,3,4,5,...|
(67,[0,2,3,4,5,6,...| 0.0|(67,[0,2,3,4,5,6,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 1.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
(67,[0,1,2,3,4,5,...| 0.0|(67,[0,1,2,3,4,5,...|
+--------------------+-----+--------------------+
only showing top 20 rows

In [49]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = scaled_data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="scaledFeatures")

# Chain indexers and tree in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = dt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
#predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g\naccuracy = %g " % ((1.0 - accuracy) ,accuracy))
print(model)
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Error = 0.0890052
accuracy = 0.910995 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_169927612042) of depth 5 with 43 nodes
Test Area Under ROC: 0.5741154610220578

In [50]:
from pyspark.ml.classification import LinearSVC

# Load training data
#training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

lsvc = LinearSVC(maxIter=5, regParam=0.1)

# Fit the model
lsvcModel = lsvc.fit(trainingData)

# Print the coefficients and intercept for linearsSVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

Coefficients: [-0.000346082141572,-0.000772222264013,-0.00149428768896,-0.00126112512875,-0.000214383443892,-0.0014774461488,-0.00423236871429,-0.00035974009528,-0.00419949666902,-0.023551923649,-0.00419949666902,-0.00152290765223,-0.00855978173939,-0.000385176531981,-0.000491021682041,-0.0121771742217,-0.00581004557025,0.0,-0.0120174204146,-0.00963040089215,-0.00653225223583,-0.0126653549301,-0.00738125118276,-0.0107112429277,-0.0123718959802,-0.00132314731169,-0.00531431578665,0.0,0.0,-0.00385208067178,-0.00777028991192,-0.00156363648898,-0.000428575861841,0.00413691466717,0.0,-0.00522712669294,0.0,0.0,-0.00551509445672,-0.00481713366118,-0.0113387294978,-0.00383643745414,-0.0107648430061,-0.0108729589164,-0.00753290043159,-0.00355940870838,-0.00979416156673,-0.00675171004339,-0.0111760540625,-0.00687775084566,-0.000368892461089,-0.00129668516563,-0.00237861662209,-0.00560739653794,-0.0261643261911,-0.00614470128903,-0.00354149625569,-0.0122964271099,-0.00119574937055,-0.00617438331584,-0.00883090049598,-0.0137244572794,-0.0113330287387,-0.00416390825317,-0.00901435687179,-0.0863562072862,-0.0144219179589]
Intercept: -0.006659965117444979

In [51]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(df_classifier)

# rescale each feature to range [min, max].
scaledData = scalerModel.transform(df_classifier)
print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
scaledData.select("features", "scaled_features").show()

Features scaled to range: [0.000000, 1.000000]
+--------------------+--------------------+
 features| scaled_features|
+--------------------+--------------------+
(67,[0,1,2,3,4,5,...|[0.52479338842975...|
(67,[0,1,2,3,4,5,...|[0.43801652892561...|
(67,[0,2,3,4,5,6,...|[0.56198347107438...|
(67,[0,1,2,3,4,5,...|[0.34297520661157...|
(67,[0,1,2,3,4,5,...|[0.30578512396694...|
(67,[0,2,3,4,5,6,...|[0.48347107438016...|
(67,[0,1,2,3,4,5,...|[0.49586776859504...|
(67,[0,2,3,4,5,6,...|[0.60330578512396...|
(67,[0,1,2,3,4,5,...|[0.47933884297520...|
(67,[0,2,3,4,5,6,...|[0.57851239669421...|
(67,[0,1,2,3,4,5,...|[0.26446280991735...|
(67,[0,2,3,4,5,6,...|[0.30165289256198...|
(67,[0,1,2,3,4,5,...|[0.69008264462809...|
(67,[0,1,2,3,4,5,...|[0.38842975206611...|
(67,[0,1,2,3,4,5,...|[0.25206611570247...|
(67,[0,1,2,3,4,5,...|[0.66115702479338...|
(67,[0,1,2,3,4,5,...|[0.34710743801652...|
(67,[0,1,2,3,4,5,...|[0.38016528925619...|
(67,[0,1,2,3,4,5,...|[0.30991735537190...|
(67,[0,1,2,3,4,5,...|[0.29752066115702...|
+--------------------+--------------------+
only showing top 20 rows

In [52]:
scaledData.show()

+--------------------+-----+--------------------+
 features|label| scaled_features|
+--------------------+-----+--------------------+
(67,[0,1,2,3,4,5,...| 0.0|[0.52479338842975...|
(67,[0,1,2,3,4,5,...| 0.0|[0.43801652892561...|
(67,[0,2,3,4,5,6,...| 0.0|[0.56198347107438...|
(67,[0,1,2,3,4,5,...| 0.0|[0.34297520661157...|
(67,[0,1,2,3,4,5,...| 0.0|[0.30578512396694...|
(67,[0,2,3,4,5,6,...| 0.0|[0.48347107438016...|
(67,[0,1,2,3,4,5,...| 0.0|[0.49586776859504...|
(67,[0,2,3,4,5,6,...| 0.0|[0.60330578512396...|
(67,[0,1,2,3,4,5,...| 0.0|[0.47933884297520...|
(67,[0,2,3,4,5,6,...| 0.0|[0.57851239669421...|
(67,[0,1,2,3,4,5,...| 1.0|[0.26446280991735...|
(67,[0,2,3,4,5,6,...| 0.0|[0.30165289256198...|
(67,[0,1,2,3,4,5,...| 0.0|[0.69008264462809...|
(67,[0,1,2,3,4,5,...| 0.0|[0.38842975206611...|
(67,[0,1,2,3,4,5,...| 0.0|[0.25206611570247...|
(67,[0,1,2,3,4,5,...| 1.0|[0.66115702479338...|
(67,[0,1,2,3,4,5,...| 0.0|[0.34710743801652...|
(67,[0,1,2,3,4,5,...| 0.0|[0.38016528925619...|
(67,[0,1,2,3,4,5,...| 0.0|[0.30991735537190...|
(67,[0,1,2,3,4,5,...| 0.0|[0.29752066115702...|
+--------------------+-----+--------------------+
only showing top 20 rows

In [53]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

#data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),(Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),(Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
#f = spark.createDataFrame(data, ["features"])
#f.show()
pca = PCA(k=5, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(scaledData)
#print(model.pc)
pcaData = model.transform(scaledData).select("pcaFeatures")
pcaData.show(1)

+--------------------+
 pcaFeatures|
+--------------------+
[-284.95067408722...|
+--------------------+
only showing top 1 row

In [54]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = scaledData.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="scaled_features")

# Chain indexers and tree in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = dt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
#predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g\naccuracy = %g " % ((1.0 - accuracy) ,accuracy))
print(model)

Test Error = 0.0552147
accuracy = 0.944785 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_e6cbea10e49a) of depth 5 with 51 nodes

In [55]:
from pyspark.ml.clustering import KMeans

# Loads data.
#dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(df_classifier)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(df_classifier)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 30740960.903948314
Cluster Centers: 
[ 1.01372514e+02 1.54611212e+00 1.00353828e+02 3.79907234e+01
 2.23472091e+02 1.00294153e+02 1.70169801e+01 2.00197709e+02
 4.48884870e+00 2.76531646e+00 4.48884870e+00 1.00724533e+02
 8.96754068e+00 1.99277878e+02 8.26943942e+00 3.07414105e-02
 2.47136829e-02 1.86859554e-02 2.65220012e-02 2.47136829e-02
 2.59192285e-02 2.35081374e-02 2.23025919e-02 2.16998192e-02
 2.10970464e-02 2.23025919e-02 2.29053647e-02 2.23025919e-02
 2.29053647e-02 2.35081374e-02 2.95358650e-02 2.53164557e-02
 2.59192285e-02 2.59192285e-02 1.62748644e-02 2.59192285e-02
 1.86859554e-02 1.62748644e-02 1.98915009e-02 1.92887281e-02
 1.62748644e-02 1.56720916e-02 1.62748644e-02 1.62748644e-02
 1.98915009e-02 1.44665461e-02 2.16998192e-02 2.04942737e-02
 1.80831826e-02 1.80831826e-02 1.80831826e-02 2.04942737e-02
 1.32610006e-02 1.38637734e-02 1.74804099e-02 1.86859554e-02
 1.44665461e-02 1.38637734e-02 1.86859554e-02 1.26582278e-02
 1.62748644e-02 1.50693189e-02 1.50693189e-02 1.44665461e-02
 1.20554551e-02 8.95117541e-01 7.18505124e-01]
[ 1.00759857e+02 1.57945042e+00 1.00516726e+02 2.32004540e+01
 1.36469654e+02 9.99360812e+01 1.71495042e+01 2.01755974e+02
 4.47013142e+00 2.76385305e+00 4.47013142e+00 9.94964158e+01
 9.11046595e+00 2.02451911e+02 7.93010753e+00 3.28554361e-02
 2.56869773e-02 3.10633214e-02 2.15053763e-02 2.21027479e-02
 2.09080048e-02 2.32974910e-02 2.38948626e-02 2.44922342e-02
 2.32974910e-02 2.15053763e-02 2.09080048e-02 2.15053763e-02
 2.03106332e-02 1.97132616e-02 1.31421744e-02 1.67264038e-02
 1.61290323e-02 1.49342891e-02 2.44922342e-02 1.49342891e-02
 2.09080048e-02 2.32974910e-02 1.97132616e-02 1.97132616e-02
 2.27001195e-02 2.32974910e-02 2.21027479e-02 2.15053763e-02
 1.79211470e-02 2.27001195e-02 1.55316607e-02 1.67264038e-02
 1.85185185e-02 1.85185185e-02 1.85185185e-02 1.55316607e-02
 2.27001195e-02 2.15053763e-02 1.73237754e-02 1.49342891e-02
 1.85185185e-02 1.85185185e-02 1.37395460e-02 1.91158901e-02
 1.55316607e-02 1.61290323e-02 1.55316607e-02 1.25448029e-02
 1.43369176e-02 9.10991637e-01 7.28195938e-01]

## Results:
### Below are the results according to the accuracy and ROC learning rates of different models, 

####                            Accuracy	ROC
#### LogisticRegression	        0.85	    --
#### Decion tree	            0.92	    0.37
#### Random Forest	            0.85        0.89
#### Gradient boosted                   
#### tree                       0.94	    0.95
#### Multilayer                         
#### Perceptron      	        0.94	    0.946

#### As per the above results we could see that all models perform better. With multilayer perceptron neural network and gradient boosted tree working better with accuracy of 94% and ROC  score of 0.95. As these are test accuracies obtained, which are calculated with the test split of 30%. Almost all the models have test accuracies of more than 85%. ROC scores with highest of 0.95 and lowest of 0.37, inferes that the model's of more ROC score learn the data faster than the models like decision trees and logistical regression with lesser ROC rates compared to other models. 

#### Also we can see that K-means algorithm under performs and couldnot learn any specific patterns in the dataset. 

#### Multilayer perceptron is a neural network that predicts well. 

#### After experimenting on different models and considering our problem statement we can infere that Tree based models works better for this situation. Tree based models show better predictability(Though the ROC score is less for decision tree) we can say that gradient boosted trees with highest accuracies and explainabilities perform better. These tree based models can be interpreted and Explained better way compared to the Multilayer perceptron and neural networks.

## Companies can make use of these models and predict whether the customer will churn or not. By leveraging the data customer can make new offers for these clients and on a whole improve the service to retain the customers.

#### References:
#### 1) https://spark.apache.org/docs/1.5.2/api/python/pyspark.ml.html
#### 2) https://towardsdatascience.com/predict-customer-churn-with-r-9e62357d47b4
#### 3) https://towardsdatascience.com/cutting-the-cord-predicting-customer-churn-for-a-telecom-company-268e65f177a5
#### 4) https://stackoverflow.com/
#### 5) Frank-Kanes-Taming-Big-Data-with-Apache-Spark-and-Python

### Challenges faced :
### We faced some challenges while implementing the models in mulltilayer perceptron and PCA analysis as the data frame transformation had to be done and also we weren't able to get the evaluation metrices for different  models. Later we had to explore various sources and finally have implemented those metrics using BinaryEvaluator
